In [3]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from dataclasses import dataclass
from scipy.stats import multivariate_normal as mvn

import vsem
from vsem_inv_prob import InvProb

In [ ]:
rng = np.random.default_rng(6435)

class TestPrior:
    def __init__(self, rng, dim: int):
        self.rng = rng
        self._dim = dim

    @property
    def dim(self):
        return self._dim

    def sample(self, n=1):
        samp = self.rng.uniform(size=(n, self.dim))

        if n == 1:
            return samp[0,:]
        else:
            return samp 

    def log_density(self, x):
        if x.ndim == 1:
            x = x.reshape(1, -1)

        return np.zeros(x.shape[0])


class TestLikelihood:
    def __init__(self, dim):
        self._dist = mvn(np.zeros(dim))

    def log_density(self, x):
        return self._dist.logpdf(x)

In [24]:
d = 2
prior = TestPrior(rng, d)
likelihood = TestLikelihood(d)

In [25]:
inv_prob = InvProb(rng, prior, likelihood)

In [37]:
inv_prob.sampler._current_state.primary["u"].shape

(1, 2)

In [27]:
test = inv_prob.sample_posterior(n_samp=10000)

ValueError: shapes of a (2, 2) and b (1, 2) are incompatible